<a href="https://colab.research.google.com/github/kedaarchakankar/CY2550/blob/master/reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reranking Retrieval Results

In this notebook, you will continue using the [Pyserini](http://pyserini.io/) library's indexing and retrieval models.  This time, however, you will get an initial set of retrieval results and then write your own reranking code to try to move relevant documents higher in the list.

As before, we start by installing the python interface. Since it calls the underlying Lucene search engine, which is written in Java, we make sure we point to an appropriate Java installation. If like Colab you don't have Java 21, uncomment the following code and run it, or whatever makes sense for your platform.

In [1]:
## Uncomment the following code to install Java 21 on Colab
!apt-get install openjdk-21-jre-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-21-openjdk-amd64/bin/java
!java -version

openjdk version "21.0.6" 2025-01-21
OpenJDK Runtime Environment (build 21.0.6+7-Ubuntu-122.04.1)
OpenJDK 64-Bit Server VM (build 21.0.6+7-Ubuntu-122.04.1, mixed mode, sharing)


In [2]:
!pip install pyserini
# You can change this to gpu if you have one.
# It's a pyserini dependency, but we won't need it until the next assignment.
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 MB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0

We initialize the searcher with a pre-built index for the Robust04 collection, which Pyserini will automatically download if it hasn't already. Note that the index takes up 1.6GB of disk.

In [3]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher.from_prebuilt_index('robust04')

lucene-inverted.disk45.20240803.36f7e3.tar.gz: 1.66GB [00:33, 52.6MB/s]                            


Now we can search for a query and inspect the results:

In [4]:
hits = searcher.search('black bear attacks', 1000)

# Prints the first 10 hits
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA092790-0015   7.06680
 2 LA081689-0039   6.89020
 3 FBIS4-16530     6.61630
 4 LA102589-0076   6.46450
 5 FT932-15491     6.25090
 6 FBIS3-12276     6.24630
 7 LA091090-0085   6.17030
 8 FT922-13519     6.04270
 9 LA052790-0205   5.94060
10 LA103089-0041   5.90650


The `IndexReaderUtils` class provides various methods to read the index directly. For example, we can fetch a raw document from the index given its `docid`:

In [43]:
from pyserini.index import LuceneIndexReader
from IPython.core.display import display, HTML

reader = LuceneIndexReader.from_prebuilt_index('robust04')

doc = reader.doc('LA092790-0015').raw()
display(HTML('<div style="font-family: Times New Roman; padding-bottom:10px">' + doc + '</div>'))

Note that the result is exactly the same as displaying the hit contents above. Given the raw text, we can obtain its analyzed form (i.e., tokenized, stemmed, stopwords removed, etc.). Here we show the first ten tokens:

In [11]:
analyzed = reader.analyze(doc)
analyzed[0:10]

['date',
 'p',
 'septemb',
 '27',
 '1990',
 'thursdai',
 'ventura',
 'counti',
 'edit',
 'p']

## Retrieving Initial Ranked Lists

We can load some standard evaluation sets such as Robust04, which contains 250 queries, or "topics" as the TREC conferences call them.

In [12]:
from pyserini.search import get_topics
topics = get_topics('robust04')
print(f'{len(topics)} queries total')

250 queries total


The topics are in a dictionary, whose keys are integers uniquely identifying each query. Each topic contains the following fields:

* `title`: TREC's term for the brief query a user might actually type;
* `description`: a longer form of the query in the form of a complete sentence; and
* `narrative`: a description of what the user is looking for and what kinds of results would be relevant or non-relevant.

In [13]:
topics[301]

{'narrative': 'A relevant document must as a minimum identify the organization and the type of illegal activity (e.g., Columbian cartel exporting cocaine). Vague references to international drug trade without identification of the organization(s) involved would not be relevant.',
 'description': 'Identify organizations that participate in international criminal activity, the activity, and, if possible, collaborating organizations and the countries involved.',
 'title': 'International Organized Crime'}

For the purpose of your experiments, we'll divide them into a development and test set.

In [14]:
dev_topics = {k:topics[k] for k in list(topics.keys())[:125]}
test_topics = {k:topics[k] for k in list(topics.keys())[125:]}

Now, we'll fetch the relevance judgments for the Robust04 queries, which TREC calls "qrels".

In [15]:
from urllib.request import urlopen

qfile = 'https://github.com/castorini/anserini-tools/blob/63ceeab1dd94c1221f29b931d868e8fab67cc25c/topics-and-qrels/qrels.robust04.txt?raw=true'
qrels = []
for line in urlopen(qfile):
  qid, round, docid, score = line.strip().split()
  qrels.append([int(qid), 0, docid.decode('UTF-8'), int(score)])
#qrels = [line.strip().split() for line in urlopen(qfile)]

Each record in the qrel contains four fields:

1. the numeric identifier of the query;
2. the round of relevance feedback, which is here always 0;
3. the identifier of a documennt that has been judged; and
4. the relevance score of that document.

In Robust04, all relevance judgments are binary, i.e., 1 or 0. Note that not all non-relevant documents are recorded. The qrel file only contains those documents the annotators actually looked at; the vast majority of documents in the collection have not been judged. In IR evaluation, we assume that unannotated documents are non-relevant.

In [16]:
qrels[0:10]

[[301, 0, 'FBIS3-10082', 1],
 [301, 0, 'FBIS3-10169', 0],
 [301, 0, 'FBIS3-10243', 1],
 [301, 0, 'FBIS3-10319', 0],
 [301, 0, 'FBIS3-10397', 1],
 [301, 0, 'FBIS3-10491', 1],
 [301, 0, 'FBIS3-10555', 0],
 [301, 0, 'FBIS3-10622', 1],
 [301, 0, 'FBIS3-10634', 0],
 [301, 0, 'FBIS3-10635', 0]]

We collect the top 1000 hists for both the dev and test sets. You

In [17]:
# Compute top-1000 lists for queries in test_topics
def topic_hits(searcher, topics, k=1000):
  hits = {}
  for topic, info in topics.items():
    print(topic, info['title'])
    hits[topic] = [(hit.docid, hit.score) for hit in searcher.search(info['title'], k)]
  return hits

dev_hits = topic_hits(searcher, dev_topics)
test_hits = topic_hits(searcher, test_topics)

350 Health and Computer Terminals
351 Falkland petroleum exploration
352 British Chunnel impact
353 Antarctica exploration
354 journalist risks
355 ocean remote sensing
356 postmenopausal estrogen Britain
357 territorial waters dispute
358 blood-alcohol fatalities
359 mutual fund predictors
360 drug legalization benefits
361 clothing sweatshops
362 human smuggling
363 transportation tunnel disasters
364 rabies
365 El Nino
366 commercial cyanide uses
367 piracy
368 in vitro fertilization
369 anorexia nervosa bulimia
370 food/drug laws
371 health insurance holistic
372 Native American casino
373 encryption equipment export
374 Nobel prize winners
375 hydrogen energy
376 World Court
377 cigar smoking
378 euro opposition
379 mainstreaming
380 obesity medical treatment
381 alternative medicine
382 hydrogen fuel automobiles
383 mental illness drugs
384 space station moon
385 hybrid fuel cars
386 teaching disabled children
387 radioactive waste
388 organic soil enhancement
389 illegal technol

## Evaluating Initial Ranked Lists



When reranking, an important metric is the _recall_ of the initial set of results. This tells us the upper bound or &ldquo;headroom&rdquo; on the improvements that reranking can achieve. If the recall in the initial ranked lists is too low, we know we need to optimize the initial retrieval model.

For this assignment, you will work with fixed initial ranked lists from pyserini's BM25 model, but it's still useful to see how much room there is for improvement during reranking.

As before, you should process the `qrels` data to find the relevant results for each query.

In [29]:
## TODO [15 points]: Compute Recall@1000 for the dev_hits and test_hits data
## and print it out.
# Step 1: Organize qrels into a dictionary: qid -> set of relevant docids
from collections import defaultdict

qrels_dict = defaultdict(set)
for qid, _, docid, score in qrels:
    if score > 0:
        qrels_dict[qid].add(docid)

def compute_recall_at_1000(hits, qrels_dict):
    total_relevant = 0
    retrieved_relevant = 0

    for qid, doc_scores in hits.items():
        retrieved_docs = set(docid for docid, _ in doc_scores)
        relevant_docs = qrels_dict[qid]

        retrieved_relevant_docs = retrieved_docs.intersection(relevant_docs)

        total_relevant += len(relevant_docs)
        retrieved_relevant += len(retrieved_relevant_docs)

    recall = retrieved_relevant / total_relevant if total_relevant > 0 else 0
    return recall

dev_recall = compute_recall_at_1000(dev_hits, qrels_dict)
test_recall = compute_recall_at_1000(test_hits, qrels_dict)

print(f"Recall@1000 on Dev Set: {dev_recall:.4f}")
print(f"Recall@1000 on Test Set: {test_recall:.4f}")

Recall@1000 on Dev Set: 0.5888
Recall@1000 on Test Set: 0.5909


For a given set of top-1000 lists, Recall@1000 will not change after reranking. What will change are ranking-based metrics like MAP and NDCG. You should compute MAP@1000 for the initial `dev_hits` and `test_hits` data.

In [30]:
## TODO [10 points]: Adapt your code from Homework 3 to compute MAP@1000 for
## the dev_hits and test_hits data and print it out.
def compute_map_at_1000(hits, qrels_dict):
    average_precisions = []

    for qid, doc_scores in hits.items():
        relevant_docs = qrels_dict[qid]
        if not relevant_docs:
            continue

        num_relevant = 0
        precision_sum = 0.0

        for rank, (docid, _) in enumerate(doc_scores[:1000], start=1):
            if docid in relevant_docs:
                num_relevant += 1
                precision = num_relevant / rank
                precision_sum += precision

        average_precision = precision_sum / len(relevant_docs)
        average_precisions.append(average_precision)

    mean_ap = sum(average_precisions) / len(average_precisions)
    return mean_ap

map_dev = compute_map_at_1000(dev_hits, qrels_dict)
map_test = compute_map_at_1000(test_hits, qrels_dict)

print(f"MAP@1000 on Dev Set: {map_dev:.4f}")
print(f"MAP@1000 on Test Set: {map_test:.4f}")

MAP@1000 on Dev Set: 0.2426
MAP@1000 on Test Set: 0.2637


## Reranking Search Results

In this final part of the assignment, you should implement a ranking function that, hopefully, improves on the baseline BM25 ranking. You may use the BM25 score for each document as input, as well as the query, of course, and any other properties of the documents you look up with the `reader` object.  After computing a new score for each candidate, re-sort the top-1000 results by your model's score.

You may use anything you've learned in this course---or in another course---to build your ranking function. For example, you could implement pseudo-relevance feedback or a relevance model, which would treat the top of each ranked list (e.g., the top 100) as if it were truly relevant and retrain model parameters. You could tune different BM25, query likelihood, or sequential dependence model parameters. You could try to learn different weights or embeddings for different fields in documents. You could use implementations of transformer language models such as BERT or SentenceBERT to score the compatibility of queries and documents. To be clear, you don't have to any of these approaches; you are free to try whatever ideas you like.

If your reranking model has tunable parameters, you should tune them on the `dev_hits` set. In the end, you will also evaluate MAP@1000 on the `test_hits` set.

**TODO**: Put any explanation of your reranking function here.

In [49]:
## TODO [70 points]: Implement a reranking function that takes a query, the
## reader, and an initial ranking and computes new scores.
## Like BM25, higher should be better.
## If you train parameters or set hyperparameters for this ranking function,
## do that here, as well.
import math

def compute_idf(reader, term, total_docs):
    """Compute IDF for a term using the index reader."""
    df = reader.get_term_counts(term)[0]
    return math.log((total_docs + 1) / (df + 1))  # Smoothed IDF

def rerank(query, reader, initial_ranking):
    reranked = []
    analyzed_query = reader.analyze(query)
    total_docs = reader.stats()['documents']

    # Precompute IDF for all query terms
    idf_scores = {term: compute_idf(reader, term, total_docs) for term in set(analyzed_query)}

    for docid, _ in initial_ranking:
        raw_doc = reader.doc(docid).raw()
        analyzed_doc = reader.analyze(raw_doc)
        doc_term_counts = defaultdict(int)
        for token in analyzed_doc:
            doc_term_counts[token] += 1

        # Compute score: sum of (TF * IDF) for query terms
        score = 0.0
        for term in analyzed_query:
            tf = doc_term_counts[term]
            idf = idf_scores.get(term, 0.0)
            score += tf * idf

        reranked.append((docid, score))

    # Sort by new score (descending)
    reranked = sorted(reranked, key=lambda x: x[1], reverse=True)
    return reranked

query_id = 350
query_text = topics[query_id]['title']  # or use 'description'

initial = dev_hits[query_id]
reranked = rerank(query_text, reader, initial)

# Print top 10 reranked results
for i, (docid, score) in enumerate(reranked[:10], start=1):
    print(f"{i}. {docid} - {score:.4f}")

1. FR940602-1-00023 - 914.1033
2. FBIS4-22970 - 468.8831
3. FT911-3933 - 202.1551
4. FT911-3935 - 195.6546
5. FBIS4-47124 - 195.6044
6. LA101290-0114 - 166.4024
7. LA031289-0009 - 155.5820
8. FR940324-2-00020 - 150.8346
9. FR941021-0-00024 - 147.1898
10. LA022189-0015 - 142.3205


In [54]:
## TODO [5 points]: Compute and print out the MAP@1000 score after reranking
## on dev_hits and test_hits.
def compute_average_precision(ranked_list, relevant_docs):
    num_relevant = 0
    total_precision = 0.0
    for i, (docid, _) in enumerate(ranked_list[:1000]):
        if docid in relevant_docs:
            num_relevant += 1
            total_precision += num_relevant / (i + 1)
    return total_precision / len(relevant_docs) if relevant_docs else 0.0

def compute_map_at_1000(hits, qrels_dict):
    average_precisions = []
    for qid, doc_scores in hits.items():
        relevant_docs = qrels_dict.get(qid, set())
        average_precisions.append(compute_average_precision(doc_scores, relevant_docs))
    return sum(average_precisions) / len(average_precisions)

# Create new reranked versions of the full dev and test sets
reranked_dev = {}
for qid in dev_hits:
    query = topics[qid]['title']
    reranked_dev[qid] = rerank(query, reader, dev_hits[qid])

reranked_test = {}
for qid in test_hits:
    query = topics[qid]['title']
    reranked_test[qid] = rerank(query, reader, test_hits[qid])

# Compute MAP@1000
map_dev = compute_map_at_1000(reranked_dev, qrels_dict)
map_test = compute_map_at_1000(reranked_test, qrels_dict)

print(f"MAP@1000 after reranking (dev): {map_dev:.4f}")
print(f"MAP@1000 after reranking (test): {map_test:.4f}")

MAP@1000 after reranking (dev): 0.0892
MAP@1000 after reranking (test): 0.1016
